In [1]:
from pathlib import Path

In [2]:
import pandas as pd
import numpy as np

In [11]:
root=Path('../').resolve()

In [12]:
root

WindowsPath('C:/Users/user/Documents/GitHub/Carkinos')

In [14]:
raw_path=root.joinpath('src','raw','GSE36133','GSE36133_RAW.quantile_normalized.tsv')
probe_path=root.joinpath('src','raw','Affy_U133plus2_probe_info.csv')
sample_path=root.joinpath('src','raw','GSE36133','CCLE.xlsx')

In [15]:
plat_info_df = pd.read_csv(probe_path.as_posix())

In [16]:
plat_info_df.head()

,PROBEID,SYMBOL,ENTREZID,GENENAME
0,1007_s_at,DDR1,780,discoidin domain receptor tyrosine kinase 1
1,1007_s_at,MIR4640,100616237,microRNA 4640
2,1053_at,RFC2,5982,"replication factor C (activator 1) 2, 40kDa"
3,117_at,HSPA6,3310,heat shock 70kDa protein 6 (HSP70B')
4,121_at,PAX8,7849,paired box 8


In [17]:
sample_info_df = pd.read_excel(sample_path.as_posix())

In [18]:
sample_info_df.head()

,Group,name,Cell_line,Source name,Primary_site,Primary_hist,Histology subtype1
0,-,GSM886835,1321N1,ECACC,central nervous system,glioma,astrocytoma
1,-,GSM886836,143B,ATCC,bone,osteosarcoma,NaN
2,-,GSM886837,22Rv1,ATCC,prostate,carcinoma,NaN
3,-,GSM886838,23132/87,DSMZ,stomach,carcinoma,adenocarcinoma
4,-,GSM886839,42-MG-BA,DSMZ,central nervous system,glioma,astrocytoma_Grade_IV


In [19]:
sample_info_df = sample_info_df.loc[
    :, ['name', 'Cell_line', 'Primary_site', 'Primary_hist']
]
sample_info_df.columns = [
    'name', 'cell_line', 'primary_site', 'primary_histology'
]

In [20]:
sample_raw_data_df = pd.read_table(raw_path.as_posix())

In [21]:
sample_raw_data_df.head()

,Unnamed: 0,GSM886835.CEL.gz,GSM886836.CEL.gz,GSM886837.CEL.gz,GSM886838.CEL.gz,GSM886839.CEL.gz,GSM886840.CEL.gz,GSM886841.CEL.gz,GSM886842.CEL.gz,GSM886843.CEL.gz,...,GSM887742.CEL.gz,GSM887743.CEL.gz,GSM887744.CEL.gz,GSM887745.CEL.gz,GSM887746.CEL.gz,GSM887747.CEL.gz,GSM887748.CEL.gz,GSM887749.CEL.gz,GSM887750.CEL.gz,GSM887751.CEL.gz
0,1007_s_at,9.069319,8.831760,10.620580,11.192532,8.990821,10.498929,9.198445,11.418360,8.707193,...,10.469033,12.210265,10.639168,11.405050,9.894988,9.697556,9.794227,11.481848,10.964601,11.633033
1,1053_at,10.203425,9.604644,10.747486,8.986573,10.272788,10.168361,9.607614,10.031919,10.191501,...,9.631346,9.236498,10.045720,9.440482,8.745399,9.460357,9.729576,9.211143,9.103648,9.113995
2,117_at,7.015166,6.897456,6.873472,6.936911,6.949447,6.953332,6.902883,6.994181,6.902269,...,6.796219,6.950180,6.995661,6.997018,6.947650,6.787864,6.894699,6.954654,6.886693,6.969157
3,121_at,8.524442,7.765468,8.355343,8.904400,8.088552,8.833986,7.968559,7.761056,7.809402,...,7.636309,7.969167,8.032827,7.958479,7.971772,7.955109,8.044165,7.983279,7.695917,8.139457
4,1255_g_at,6.816810,6.693465,6.710513,6.592209,6.716282,6.837487,6.747872,6.676317,6.746093,...,6.819875,6.718008,6.706068,6.775780,6.746609,7.091192,6.762120,6.593542,6.706902,6.754710


In [22]:
sample_raw_data_df.index = sample_raw_data_df['Unnamed: 0']
sample_raw_data_df.index.name = None

In [ ]:
sample_raw_data_df = sample_raw_data_df.iloc[:, 1:]

In [ ]:
sample_raw_data_df.columns = [col[:-len('.CEL.gz')] for col in sample_raw_data_df.columns]
sample_raw_data_df = sample_raw_data_df.loc[:, sample_info_df['name']]
sample_raw_data_df = sample_raw_data_df.reindex(pd.unique(plat_info_df.PROBEID))

In [ ]:
npy_path=root.joinpath('src','GSE36133.npy')
with (npy_path).open('wb') as f:
    np.save(f, sample_raw_data_df.values.astype(np.float32))